In [1]:
#!pip install nl4py
!pip show nl4py
from os import environ
environ["netlogo_path"]="/opt/netlogo"

if environ.get('netlogo_path') is not None:
    netlogo_path = environ.get('netlogo_path')
else:
    netlogo_path = "C:/Program Files/NetLogo 6.0.2/"

import nl4py
# Replace argument to startServer(...) with the location of your NetLogo installation 
nl4py.startServer(netlogo_path)
import time
def ethnocentrism_setup_callback(data):
    # Same netlogo commands as used for the PyNetLogo evaluation
    setup_commands = []
    setup_commands.append("random-seed " + str(data))
    #setup_commands.append("set mutation-rate 0.005")
    setup_commands.append("set death-rate 0.1")
    setup_commands.append("set immigrants-per-day 1")
    setup_commands.append("set immigrant-chance-cooperate-with-same 0.5")
    setup_commands.append("set immigrant-chance-cooperate-with-different 0.5")
    setup_commands.append("set initial-PTR 0.12")
    setup_commands.append("set cost-of-giving 0.01")
    setup_commands.append("set gain-of-receiving 0.03")
    setup_commands.append('setup-empty')
    return setup_commands

Name: NL4Py
Version: 0.5.10
Summary: A NetLogo connector for Python.
Home-page: https://github.com/chathika/NL4Py
Author: Chathika Gunaratne
Author-email: chathikagunaratne@gmail.com
License: GPL
Location: /home/social-sim/miniconda2/envs/nl4py/lib/python3.7/site-packages
Requires: numpy, psutil, py4j, matplotlib, pandas
Required-by: 


In [2]:
def measureExecutionTime(runsNeeded):
    startTime = int(round(time.time() * 1000))
    # Make sure we start n headless workspaces to compare to n IPCluster engines running PyNetLogo
    modelname = "./Models/Ethnocentrism.nlogo"
    measures = ['ticks','count turtles']
    data = list(range(runsNeeded))
    results = nl4py.runExperiment(model_name = modelname, callback=ethnocentrism_setup_callback,data=data,reporters=measures, start_at_tick=0,interval=1,stop_at_tick=ticksNeeded,go_command="go",num_procs=multiprocessing.cpu_count())
    stopTime = int(round(time.time() * 1000))
    #print(results)
    executionTime = stopTime - startTime
    return executionTime, results

In [3]:
import os
outputFile = '../output/5.3_output.csv'
out = open (outputFile, "a+")

In [4]:
for runs in [2000]:
    executionTime, results = measureExecutionTime(runs)
    print("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime))
    out.write("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime) + "\n")
    nl4py.deleteAllHeadlessWorkspaces()
    out.flush()

AttributeError: module 'nl4py' has no attribute 'runExperiment'

In [1]:
results[:,1]


NameError: name 'results' is not defined

In [ ]:
out.close()
nl4py.stopServer()

In [1]:
#!pip install nl4py
import numpy as np
import pandas as pd
import pyNetLogo
import os
import nl4py
import ipyparallel
from os import environ
os.environ["netlogo_path"] = "/opt/netlogo/"
if os.environ.get('netlogo_path') is not None:
    netlogo_path = os.environ.get('netlogo_path')
else:
    netlogo_path = "C:/Program Files/NetLogo 6.0.2/"
nl4py.startServer(netlogo_path)
# Start IPCluster with 8 engines now through the command line using the following command:
# IPCluster start -n 8
# 8 engines are used for fair comparison against 8 parallel headless workspaces through NL4Py
client = ipyparallel.Client()
print(client.ids)
direct_view = client[:]
import os
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]


<AsyncResult: _push>

In [2]:
%%px 
#Jupyter notebook decorator to make the notebook cell parallelizable
import os
os.chdir(cwd) # change to the current directory to the IPCluster engines
nl4py.startServer("/opt/netlogo")

#print(netlogo_path)
# Replace argument netlogo_home with the location of your NetLogo installation 
ws = nl4py.newNetLogoHeadlessWorkspace()
ws.openModel('../models/Wolf Sheep Predation.nlogo')

In [9]:
def simulate(data):
    #workspace_.command("stop")
    #workspace_.openModel("../models/Wolf Sheep Predation.nlogo")    
    # Same netlogo commands as used for the PyNetLogo evaluation
    ws.command("random-seed " + str(data))
    ws.command("set initial-number-sheep 150")
    ws.command("set initial-number-wolves 150")
    ws.command("set grass-regrowth-time 50")
    ws.command("set sheep-gain-from-food 25")
    ws.command("set wolf-gain-from-food 50")
    ws.command("set sheep-reproduce 10")
    ws.command("set wolf-reproduce 10")
    ws.command("set show-energy? false")
    ws.command('set model-version "sheep-wolves-grass"')
    ws.command('setup')
    ws.scheduleReportersAndRun(["ticks",'count sheep','count wolves'], 0,1,100,"go")  
    ws.awaitScheduledReporterResults()

In [10]:
import time
for runs in [1000,2000,3000]:
    startTime = int(round(time.time() * 1000))
    # Run the simulations and measure execution time
    #try:
    #The following failsafe had to be added as PyNetLogo fails due to duplicated temporary log file names that are left behind from previous runs
    """test = os.listdir(os.getcwd())
    for item in test:
        if item.endswith(".txt"):
            os.remove(os.path.join(os.getcwd(), item))"""
    ## end failsafe
    ## load balance view failed to run consistently. Therefore, reverted to using direct view in order to get PyNetLogo running on an IPCluster
    results = direct_view.map_sync(simulate, range(runs))
    #print(results)
    #except Exception as e: 
    #    print(e)
    #    pass
    stopTime = int(round(time.time() * 1000))
    executionTime = stopTime - startTime
    print("nl4py,repeatReporters," + str(runs) + "," + str(executionTime))
    #out.write("nl4py,repeatReporters," + str(runs) + "," + str(executionTime) + "\n")
    #out.flush()

nl4py,repeatReporters,1000,19069
nl4py,repeatReporters,2000,40929
nl4py,repeatReporters,3000,58711
